In [105]:
# !pip install webdriver_manager
# !pip install pandas
# !pip install selenium
# !pip install pillow

In [ ]:
# import
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import requests
import csv
import os
from PIL import Image
from io import BytesIO

In [8]:
departments = ["신경과", "신경외과", "신장내과", "심장내과", "심장혈관흉부외과",
               "안과", "알레르기내과", "영상의학과", "외과", "응급의학과",
               "이비인후-두경부외과", "재활의학과", "정신건강의학과", "정형외과",
               "종양혈액내과", "직업환경의학과", "진단검사의학과", "치과", "피부과",
               "한의과", "핵의학과", "호흡기내과"]

all_doctor_info_list = []

for department in departments:
    
    doctor_info_list = []
    
    # 1. chrome driver 띄우기
    chrome_options = Options() 
    chrome_options.add_experimental_option("detach", True)

    url = 'https://www.ish.or.kr/main/part/list.do' # 가톨릭관동대학교 국제성모병원
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(10)
    driver.get(url) 
    
    # 2. 진료과별로 데이터 가져오기
    # 2.1. 과 검색
    search_box = driver.find_element(By.ID, 'v_keyword')
    search_box.clear()
    search_box.send_keys(department)

    search_button = driver.find_element(By.CLASS_NAME, 'srch_btn')
    search_button.click()

    # 2.2. 의료진 버튼 click
    id_pattern = re.compile(r'part\d+')
    elements = driver.find_elements(By.XPATH, '//*[@id]')

    for element in elements:
        element_id = element.get_attribute('id')
        if id_pattern.match(element_id):
            mouse = element
            break

    css_selector = f'a[title="{department}"]'

    button = driver.find_element(By.CSS_SELECTOR, css_selector)
    act = ActionChains(driver)
    act.move_to_element(mouse).click(button).perform()

    time.sleep(1)

    # 3. 데이터 가져오기

    # 3.1. html 파싱하기

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup

    # 3.2. 의료진 정보 가져오기
    doctor_info_list = []

    for i in range(1, 200):
        doctor_id = f"doctor_info{i}"
        doctor_elem = soup.find('li', {'id': doctor_id})
        
        if doctor_elem:
            part_elem = doctor_elem.find('p', class_='part')
            name_elem = doctor_elem.find('p', class_='name')
            img_elem = doctor_elem.find('img', {'src': True})
            clinic_elem = doctor_elem.find('p', class_='clinic')
            
            if part_elem and name_elem and img_elem and clinic_elem:
                dptm = part_elem.text.strip()  # '신경과'
                name = name_elem.text.strip().split()[0]  # '구본대'
                image_name = img_elem['src'].split('/')[-1]  # '545828s.png'
                position = name_elem.text.strip().split()[-1]  # '교수'
                clinic = clinic_elem.text.strip()  # '퇴행성 척추질환(목, 허리 디스크), ...'
                img_src = "https://www.ish.or.kr" + img_elem['src']

                # 3.3. 이미지 다운로드
                response = requests.get(img_src)
                if response.status_code == 200:
                    image_data = response.content
                    image = Image.open(BytesIO(image_data))
                    image.save(f"02/images/{image_name}")  

                # 3.4. 의료진 데이터 저장
                doctor_info = {
                    'dptm': dptm,
                    'name': name,
                    'image': image_name, 
                    'position': position,
                    'clinic': clinic,
                }
                doctor_info_list.append(doctor_info)
                
                all_doctor_info_list.extend(doctor_info_list)
                
                   
    driver.quit()

# 4. csv로 내보내기
with open("data/02/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'image', 'position', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in all_doctor_info_list:
        writer.writerow(doctor_info)